In [23]:
using ForwardDiff
using Distances
using LinearAlgebra
using ReverseDiff
using AutoGrad

In [ ]:
LR = 0.1 # learning rate
EPOCHS = 1000 # number of epochs
DIM = 250 # dimensionality of embeddings


In [ ]:
function radius(x)
    return x[end]
end

function centerpoint(x)
    return x[1:end-1]
end


In [ ]:
function loss1(c, d) # loss for normal form 1
    return max(0, euclidean(centerpoint(c), centerpoint(d)) + abs(radius(c))-abs(radius(d)))
end

function loss2(c1, c2, d) # loss for c1 and c2 SubClassOf: d
    dist = euclidean(centerpoint(c1), centerpoint(c2))
    if dist > radius(c1) + radius(c2)
        return abs(radius(c2)) + abs(radius(c2)) # no solution, circles are separate
    elseif dist < abs(radius(c1)-radius(c2)) # no solution, one circle contained in other
        if radius(c1) <= radius(c2)
            return loss1(c1, d)
        else
            return loss1(c2, d)
        end
    elseif dist == 0 && radius(c1) == radius(c2) ## circles are coincident
        return loss1(c1, d)
    else
        a = (radius(c1)^2 - radius(c2)^2 + dist^2)/(2 * dist)
        rad = sqrt(radius(c1)^2 - a^2)
        cent = centerpoint(c1) + a * (centerpoint(c2) - centerpoint(c1)) / dist
        return loss1(vcat(cent, rad),d)
    end
end

function loss3(c, d, r) # normal form 3
    return max(0, euclidean(centerpoint(c)-r, centerpoint(d)) + abs(radius(d)) - abs(radius(c)))
end

function loss4(c, d, r) # normal form 4
    return max(0, euclidean(centerpoint(c)+r, centerpoint(d)) + abs(radius(d)) - abs(radius(c)))
end


In [ ]:
classes = Dict()
relations = Dict()
ccount = 1
rcount = 1

function filldict(x)
    global ccount
    if haskey(classes, x)
        return classes[x]
    else
        classes[x] = ccount
        ccount += 1
        return ccount - 1
    end
end

function rfilldict(x)
    global rcount
    if haskey(relations, x)
        return relations[x]
    else
        relations[x] = rcount
        rcount += 1
        return rcount - 1
    end
end


In [ ]:
nf1 = Set()
nf2 = Set()
nf3 = Set()
nf4 = Set()


In [ ]:
# Reading the file of normalized axioms (generated by Normalize.groovy)
open("go-normalized.txt") do file
    for line in eachline(file)
        if (startswith(line, "SubClassOf")) # ignore subproperty axioms
            if occursin(r"ObjectIntersectionOf", line) # normal form 2
                m = match(r"(<.*>).*(<.*>).*(<.*>)", line) # 3 captures: C and D SubClassOf: E
                if m != nothing
                    c, d, e = map(filldict, m.captures)
                    push!(nf2, (c,d,e))
                end
            elseif occursin(r"SubClassOf.<[^\s]*>[\s]*<[^\s]*>.$", line) # normal form 1
                m = match(r"(<.*>).*(<.*>)", line) # 2 captures: C SubClassOf: D
                c,d = map(filldict, m.captures)
                push!(nf1, (c,d))
            elseif occursin(r"SubClassOf.ObjectSomeValuesFrom", line)  # normal form 4
                m = match(r"(<.*>).*(<.*>).*(<.*>)", line) # 3 captures: R some C SubClassOf: D
                r,c,d = m.captures
                c = filldict(c)
                d = filldict(d)
                r = rfilldict(r)
                push!(nf4, (c,d,r))
            elseif occursin(r"SubClassOf.<[^\s]*>[\s]*ObjectSomeValuesFrom", line)  # normal form 3
                m = match(r"(<.*>).*(<.*>).*(<.*>)", line) # 3 captures: C SubClassOf: R some D
                c,r,d = m.captures
                c = filldict(c)
                d = filldict(d)
                r = rfilldict(r)
                push!(nf3, (c,d,r))
            end
        end
    end
end

# there are now rcount-1 relations and ccount-1 classes; generate random embeddings
cvec = rand(ccount-1, DIM + 1)
rvec = rand(rcount-1, DIM)


In [ ]:
function l1(cvec)
    global nf1
    res = 0
    for i in nf1
        c1 = i[1]
        c2 = i[2]
        cv1 = cvec[c1, :]
        cv2 = cvec[c2, :]
        gradient = ReverseDiff.gradient((x,y) -> loss1(x,y), (cv1, cv2))
        cvec[c1, :] -= gradient[1] * LR  # updating online
        cvec[c2, :] -= gradient[2] * LR
    end
end

In [ ]:
for iter in 1:EPOCHS
ll1 = 0.0
ll2 = 0.0
ll3 = 0.0
ll4 = 0.0

for i in nf1
    c1 = i[1]
    c2 = i[2]
    
    cv1 = cvec[c1, :]
    cv2 = cvec[c2, :]
    
    ll1 += loss1(cv1, cv2)

    gradient = ReverseDiff.gradient((x,y) -> loss1(x,y), (cv1, cv2))
    cvec[c1, :] -= gradient[1] * LR  # updating online
    cvec[c2, :] -= gradient[2] * LR
end
println("Total loss for NF1 at epoch $iter is $ll1.")
    
for i in nf2
    c1 = i[1]
    c2 = i[2]
    c3 = i[3]

    cv1 = cvec[c1, :]
    cv2 = cvec[c2, :]
    cv3 = cvec[c3, :]
    
    ll2 += loss2(cv1, cv2, cv3)

    gradient = ReverseDiff.gradient((x,y,z) -> loss2(x,y,z), (cv1, cv2, cv3))
    cvec[c1, :] -= gradient[1] * LR  # learn online
    cvec[c2, :] -= gradient[2] * LR
    cvec[c3, :] -= gradient[3] * LR
end
println("Total loss for NF2 at epoch $iter is $ll2.")

for i in nf3
    c1 = i[1]
    c2 = i[2]
    r = i[3]

    cv1 = cvec[c1, :]
    cv2 = cvec[c2, :]
    rv = rvec[r, :]
    
    ll3 += loss3(cv1, cv2, rv)
    gradient = ReverseDiff.gradient((x,y,z) -> loss3(x,y,z), (cv1, cv2, rv))
    cvec[c1, :] -= gradient[1] * LR
    cvec[c2, :] -= gradient[2] * LR
    rvec[r, :] -= gradient[3] * LR

end
println("Total loss for NF3 at epoch $iter is $ll3.")

for i in nf4
    c1 = i[1]
    c2 = i[2]
    r = i[3]

    cv1 = cvec[c1, :]
    cv2 = cvec[c2, :]
    rv = rvec[r, :]
    ll4 += loss4(cv1, cv2, rv)


    gradient = ReverseDiff.gradient((x,y,z) -> loss4(x,y,z), (cv1, cv2, rv))
    cvec[c1, :] -= gradient[1] * LR
    cvec[c2, :] -= gradient[2] * LR
    rvec[r, :] -= gradient[3] * LR
end
println("Total loss for NF4 at epoch $iter is $ll4.")

end


In [ ]:
c1 = cvec[1, :]
c2 = cvec[2, :]
loss1(c1, c2)
ReverseDiff.gradient((x,y) -> loss1(x, y), (c1, c2))
grad(loss1)